In [1]:
# Setup working directory and paths
import sys
import os

# Change to the project root directory
project_root = '/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/'
os.chdir(project_root)
print(f"Changed working directory to: {os.getcwd()}")

# Add paths to sys.path
src_path = os.path.join(project_root, 'src')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
if src_path not in sys.path:
    sys.path.insert(0, src_path)


print("Setup complete!")

Changed working directory to: /Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management
Setup complete!


In [18]:
import sc_pipe_management.wrangler_utils.analysis_set_setup_utils as ana_tools
import sc_pipe_management.wrangler_utils.constant as ana_const
import sc_pipe_management.igvf_and_terra_api_tools as api_tools
import json
import datetime
import pandas as pd
import importlib

In [17]:
importlib.reload(ana_tools)

<module 'sc_pipe_management.wranger_utils.analysis_set_setup_utils' from '/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/src/sc_pipe_management/wranger_utils/analysis_set_setup_utils.py'>

### Set up APIs

In [4]:
# Set up API (or 'https://api.staging.igvf.org')
igvf_endpt = 'prod'
igvf_keys = api_tools.set_up_api_keys(igvf_endpoint=igvf_endpt)
igvf_client_api = api_tools.get_igvf_client_auth(igvf_api_keys=igvf_keys, igvf_endpoint=igvf_endpt)
iu_conn_api = api_tools.get_igvf_utils_connection(igvf_api_keys=igvf_keys, igvf_utils_mode=igvf_endpt)

2025-10-03 16:45:47,082:iu_debug:	submission=False: In non-submission mode.


### Set up placeholder analysis sets

#### Step 1: IGVF portal query

In [4]:
# Set up query info
lab_id = '/labs/jason-buenrostro/'
award_id = '/awards/HG011986/'
preferred_assay_titles = ['SHARE-seq']
statues = ['released']
sample_names = ['colon', 'colonic epithelium', 'colon epithelial cell']
query_limit = 'all'

In [5]:
# Set up query params
query_params = ana_tools.generate_filtered_fields(lab_id=lab_id,
                                                  award_id=award_id,
                                                  preferred_assay_titles=preferred_assay_titles,
                                                  statues=statues,
                                                  sample_names=sample_names)
query_params

{'status': ['released'],
 'lab.@id': '/labs/jason-buenrostro/',
 'award.@id': '/awards/HG011986/',
 'preferred_assay_titles': ['SHARE-seq'],
 'audit.NOT_COMPLIANT.category!': ['missing sequence specification',
  'missing barcode replacement file',
  'missing read names',
  'missing barcode onlist'],
 'audit.ERROR.category!': ['upload status not validated',
  'unexpected barcode onlist',
  'inconsistent sequence specifications',
  'inconsistent preferred assay titles'],
 'samples.sample_terms.term_name': ['colon',
  'colonic epithelium',
  'colon epithelial cell']}

In [6]:
# Query the website for measurement sets meeting the criteria
query_res = ana_tools.query_for_measets(filtered_fields=query_params,
                                        igvf_client_api=igvf_client_api,
                                        limit=query_limit)

In [7]:
len(query_res)

39

#### Step 2: Generate input_file_sets arrays to make new analysis sets

In [18]:
# Make input file sets arrays
new_input_file_sets_arrays = ana_tools.get_all_input_file_sets(query_res=query_res,
                                                               igvf_client_api=igvf_client_api)

In [19]:
len(new_input_file_sets_arrays)

1

In [10]:
# Hand check a few just to be sure
for pair in new_input_file_sets_arrays:
    print(pair)

['/measurement-sets/IGVFDS1130NULC/', '/measurement-sets/IGVFDS1680VACI/', '/measurement-sets/IGVFDS3212RENM/', '/measurement-sets/IGVFDS4205JWPE/']


#### Step 3: Create payload and submit to portal

In [15]:
# Create payload
new_analysis_sets_accessioned = ana_tools.post_all_anasets_to_portal(all_input_file_sets=new_input_file_sets_arrays,
                                                                     lab=lab_id,
                                                                     award=award_id,
                                                                     igvf_utils_api=iu_conn_api,
                                                                     igvf_client_api=igvf_client_api)

2025-09-02 15:28:34,567:iu_debug:	
IN post().


DEBUG:iu_debug:
IN post().


2025-09-02 15:28:34,570:iu_debug:	Validating the payload against the schema


DEBUG:iu_debug:Validating the payload against the schema


2025-09-02 15:28:34,591:iu_debug:	<<<<<< POST analysis_set record jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline To IGVF database with URL https://api.staging.igvf.org/analysis_set and this payload:

{
  "aliases": [
    "jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline"
  ],
  "award": "/awards/HG011986/",
  "file_set_type": "intermediate analysis",
  "input_file_sets": [
    "/measurement-sets/IGVFDS1130NULC/",
    "/measurement-sets/IGVFDS1680VACI/",
    "/measurement-sets/IGVFDS3212RENM/",
    "/measurement-sets/IGVFDS4205JWPE/"
  ],
  "lab": "/labs/jason-buenrostro/",
  "uniform_pipeline_status": "preprocessing"
}




...

DEBUG:iu_debug:{'@type': ['HTTPConflict', 'Error'], 'status': 'error', 'code': 409, 'title': 'Conflict', 'description': 'There was a conflict when trying to complete your request.', 'detail': "Keys conflict: [('alias', 'jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline')]"}


2025-09-02 15:28:35,029:iu_debug:	>>>>>>GET jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline From DACC with URL https://api.staging.igvf.org/jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline/?format=json


DEBUG:iu_debug:>>>>>>GET jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline From DACC with URL https://api.staging.igvf.org/jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline/?format=json


2025-09-02 15:28:35,409:iu_debug:	Will not POST 'jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline' since it already exists with aliases '['jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline']'.


ERROR:iu_debug:Will not POST 'jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline' since it already exists with aliases '['jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline']'.
ERROR:iu_error:Will not POST 'jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline' since it already exists with aliases '['jason-buenrostro:Sample-IGVFSM7017UXAA_single-cell-uniform-pipeline']'.


In [16]:
len(new_analysis_sets_accessioned)

1

In [17]:
new_analysis_sets_accessioned

[AnaSetCreationResult(accession='TSTDS14505600', input_file_sets=['/measurement-sets/IGVFDS1130NULC/', '/measurement-sets/IGVFDS1680VACI/', '/measurement-sets/IGVFDS3212RENM/', '/measurement-sets/IGVFDS4205JWPE/'], errors=None)]

#### Step 4: Output file to a txt

In [18]:
ana_tools.write_anaset_accs_to_file(portal_post_res=new_analysis_sets_accessioned,
                                    lab_id=lab_id,
                                    preferred_assay_titles=preferred_assay_titles,
                                    additional_desc='colon',
                                    release_statuses=statues)

'/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Buenrostro/buenrostro_released_SHARE-seq_colon_20250902/buenrostro_released_SHARE-seq_colon_20250902.txt'

##

### Utils #0: Sandbox

In [20]:
foo_inputs = [
    ['TSTDS74957974', 'TSTDS31000813'],
    ['TSTDS20638124', 'TSTDS16685999'],
    ['TSTDS68628693', 'TSTDS69083547']
]

In [22]:
foo = ana_tools.post_all_anasets_to_portal(all_input_file_sets=foo_inputs,
                                           lab=lab_id,
                                           award=award_id,
                                           igvf_utils_api=iu_conn_api,
                                           igvf_client_api=igvf_client_api)

2025-09-02 15:37:41,111:iu_debug:	
IN post().


DEBUG:iu_debug:
IN post().


2025-09-02 15:37:41,113:iu_debug:	Validating the payload against the schema


DEBUG:iu_debug:Validating the payload against the schema


2025-09-02 15:37:41,139:iu_debug:	<<<<<< POST analysis_set record jason-buenrostro:Sample-_single-cell-uniform-pipeline To IGVF database with URL https://api.sandbox.igvf.org/analysis_set and this payload:

{
  "aliases": [
    "jason-buenrostro:Sample-_single-cell-uniform-pipeline"
  ],
  "award": "/awards/HG011986/",
  "file_set_type": "intermediate analysis",
  "input_file_sets": [
    "TSTDS74957974",
    "TSTDS31000813"
  ],
  "lab": "/labs/jason-buenrostro/",
  "uniform_pipeline_status": "preprocessing"
}




...

DEBUG:iu_debug:{'@type': ['HTTPConflict', 'Error'], 'status': 'error', 'code': 409, 'title': 'Conflict', 'description': 'There was a conflict when trying to complete your request.', 'detail': "Keys conflict: [('alias', 'jason-buenrostro:Sample-_single-cell-uniform-pipeline')]"}


2025-09-02 15:37:49,925:iu_debug:	>>>>>>GET jason-buenrostro:Sample-_single-cell-uniform-pipeline From DACC with URL https://api.sandbox.igvf.org/jason-buenrostro:Sample-_single-cell-uniform-pipeline/?format=json


DEBUG:iu_debug:>>>>>>GET jason-buenrostro:Sample-_single-cell-uniform-pipeline From DACC with URL https://api.sandbox.igvf.org/jason-buenrostro:Sample-_single-cell-uniform-pipeline/?format=json


2025-09-02 15:37:55,513:iu_debug:	Will not POST 'jason-buenrostro:Sample-_single-cell-uniform-pipeline' since it already exists with aliases '['jason-buenrostro:Sample-_single-cell-uniform-pipeline']'.


ERROR:iu_debug:Will not POST 'jason-buenrostro:Sample-_single-cell-uniform-pipeline' since it already exists with aliases '['jason-buenrostro:Sample-_single-cell-uniform-pipeline']'.
ERROR:iu_error:Will not POST 'jason-buenrostro:Sample-_single-cell-uniform-pipeline' since it already exists with aliases '['jason-buenrostro:Sample-_single-cell-uniform-pipeline']'.


In [23]:
foo

[AnaSetCreationResult(accession='TSTDS41499750', input_file_sets=['TSTDS74957974', 'TSTDS31000813'], errors=None),
 AnaSetCreationResult(accession='TSTDS41499750', input_file_sets=['TSTDS20638124', 'TSTDS16685999'], errors=None),
 AnaSetCreationResult(accession='TSTDS41499750', input_file_sets=['TSTDS68628693', 'TSTDS69083547'], errors=None)]

### Utils #1: Check if a list of lab generated analysis sets have bad audits

In [19]:
def get_input_measets(analysis_set_obj: dict, igvf_utils_api):
    measet_ids = []
    for fileset_obj in analysis_set_obj.get('input_file_sets'):
        if fileset_obj['@id'].startswith('/measurement-sets/'):
            measet_ids.append(fileset_obj['@id'])
    return measet_ids


def get_measet_objs(analysis_set_acc: str, igvf_utils_api):
    """Get a list of measurement set objects from the input_file_sets array."""
    analysis_set_obj = igvf_utils_api.get(analysis_set_acc)
    return [igvf_utils_api.get(measet_id) for measet_id in get_input_measets(analysis_set_obj, igvf_utils_api)]


def check_specific_audits(measet_obj: dict):
    """Check if the measurement sets have audits from a specific list."""
    relevant_audits = ana_const.EXCLUDED_ERROR_AUDITS + ana_const.EXCLUDED_NONCOMP_AUDITS
    all_audits = measet_obj.get('audit', {})
    if all_audits:
        bad_audits = all_audits.get('ERROR', []) + all_audits.get('NOT_COMPLIANT', [])
        if bad_audits:
            return any([err['category'] in relevant_audits for err in bad_audits])
        else:
            return False
    else:
        return False


def check_bad_audits(analysis_set_acc: str, igvf_utils_api):
    """Run bad audit check on a single analysis set"""
    measet_objs = get_measet_objs(analysis_set_acc, igvf_utils_api)
    if any([check_specific_audits(measet_obj) for measet_obj in measet_objs]):
        return analysis_set_acc


def check_bad_audits_all_anasets(analysis_set_accs: list, igvf_utils_api):
    results = [check_bad_audits(analysis_set_acc, igvf_utils_api) for analysis_set_acc in analysis_set_accs]
    return [result for result in results if result is not None]

In [16]:
# Lab created AnaSet accs
lab_submitted_analysis_sets_file = '/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Buenrostro/buenrostro_bonemarrow_share-seq_20251002/buenrostro_bonemarrow_share-seq_20251002.txt'

# Read into a list
with open(lab_submitted_analysis_sets_file) as f:
    analysis_sets_to_check = [entry.strip() for entry in f.readlines() if not entry.startswith("#")]
len(analysis_sets_to_check)

25

In [20]:
# Get analysis sets with audits not compatible with scPipe
anasets_with_bad_audits = check_bad_audits_all_anasets(analysis_set_accs=analysis_sets_to_check,
                                                       igvf_utils_api=iu_conn_api)

2025-10-03 20:53:05,403:iu_debug:	>>>>>>GET IGVFDS8266AMFG From DACC with URL https://api.data.igvf.org/IGVFDS8266AMFG/?format=json
2025-10-03 20:53:05,699:iu_debug:	>>>>>>GET measurement-sets/IGVFDS7133CHRI From DACC with URL https://api.data.igvf.org/measurement-sets/IGVFDS7133CHRI/?format=json
2025-10-03 20:53:05,954:iu_debug:	>>>>>>GET measurement-sets/IGVFDS5266ONOB From DACC with URL https://api.data.igvf.org/measurement-sets/IGVFDS5266ONOB/?format=json
2025-10-03 20:53:06,210:iu_debug:	>>>>>>GET IGVFDS5502QHAX From DACC with URL https://api.data.igvf.org/IGVFDS5502QHAX/?format=json
2025-10-03 20:53:06,532:iu_debug:	>>>>>>GET measurement-sets/IGVFDS8001MFIY From DACC with URL https://api.data.igvf.org/measurement-sets/IGVFDS8001MFIY/?format=json
2025-10-03 20:53:06,786:iu_debug:	>>>>>>GET measurement-sets/IGVFDS3237WAED From DACC with URL https://api.data.igvf.org/measurement-sets/IGVFDS3237WAED/?format=json
2025-10-03 20:53:07,038:iu_debug:	>>>>>>GET IGVFDS0877TXSK From DACC wit

In [22]:
len(anasets_with_bad_audits)

25

### Utils #2: Rm processed data

In [3]:
with open('/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Mortazavi/mortazavi_parse_trem2-F1_07222025/mortazavi_parse_trem2-F1_07222025.txt') as f:
    mortazavi_trem2F1_plate_anasets = [entry.strip() for entry in f.readlines()]
print(len(mortazavi_trem2F1_plate_anasets))

132


In [8]:
# Bad analysis sets
input_params_tbl = pd.read_csv('/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/input/07222025/mortazavi_parse_trem2-F1_07222025/single-cell_uniform_pipeline_input_table_2025-07-22-18-06-26_withGSpath.tsv', sep='\t')
bad_analysis_sets = input_params_tbl[input_params_tbl['possible_errors'].str.startswith('Error', na=False)]['analysis_set_acc'].to_list()

In [15]:
# Good list
good_list = list(set(mortazavi_trem2F1_plate_anasets) - set(bad_analysis_sets))
len(good_list)

119

In [16]:
# Runnable 
updated_output_file = '/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Mortazavi/mortazavi_parse_trem2-F1_07222025/BAD_mortazavi_parse_trem2-F1_07222025.txt'

with open(updated_output_file, 'w') as f:
    for accession in bad_analysis_sets:
        f.writelines(f'{accession}\n')

### Utils #3: Count measurement sets

In [5]:
def calc_input_seqfile_sizes_and_numbers(analysis_sets: list[str], igvf_client_api):
    """Check the total runnable seqfile size and file count"""
    total_file_sizes_by_anasets = {}
    for anaset in analysis_sets:
        curr_file_size = 0
        curr_file_cnt = 0
        anaset_obj = igvf_client_api.analysis_sets(anaset).graph[0]
        measurement_sets_objs = [igvf_api.get_by_id(fileset).actual_instance for fileset in anaset_obj.input_file_sets if fileset.startswith('/measurement-sets/')]
        for measurement_set_obj in measurement_sets_objs:
            seq_files = [file for file in measurement_set_obj.files if file.startswith('/sequence-files/')]
            curr_file_cnt += len(seq_files)
            for seq_file in seq_files:
                seq_file_obj = igvf_api.get_by_id(seq_file).actual_instance
                if seq_file_obj.read_names:
                    curr_file_size += seq_file_obj.file_size
        total_file_sizes_by_anasets[anaset] = [str(round((curr_file_size / (1024 **3)), 2))+'GB', curr_file_cnt]
    return total_file_sizes_by_anasets

def calc_input_measets_cnts(analysis_set_accessions: list[str], igvf_client_api):
    output_res = []
    for anaset_acc in analysis_set_accessions:
        anaset_obj = igvf_client_api.get_by_id(f'/analysis-sets/{anaset_acc}').actual_instance
        input_measet_ids = [file_set_id for file_set_id in anaset_obj.input_file_sets if file_set_id.startswith('/measurement-sets/')]
        output_res.append((anaset_acc, len(input_measet_ids)))
    return pd.DataFrame(output_res, columns=['Analysis Set Accession', 'Total Input Measurement Sets'])

In [12]:
# Starter analysis set list file
query_anaset_acc_setup_file = '/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Satpathy/satpathy_MULTI-seq_20250716/satpathy_MULTI-seq_20250716.txt'

# Get the list
with open(query_anaset_acc_setup_file) as f:
    query_anaset_acc_list = [entry.strip() for entry in f.readlines()]
len(query_anaset_acc_list)

48

In [14]:
# Count
count_res = calc_input_measets_cnts(query_anaset_acc_list, igvf_client_api)
count_res

,Analysis Set Accession,Total Input Measurement Sets
0,IGVFDS8997FOWM,2
1,IGVFDS2543QAMJ,2
2,IGVFDS6955JCTX,2
3,IGVFDS4450VKHD,2
4,IGVFDS4364HEPW,2
5,IGVFDS0443IFQY,2
6,IGVFDS8327WMFU,2
7,IGVFDS3174HEKS,2
8,IGVFDS0706KKYL,2
9,IGVFDS1101ACFG,2


In [15]:
count_res.sum()

Analysis Set Accession          IGVFDS8997FOWMIGVFDS2543QAMJIGVFDS6955JCTXIGVF...
Total Input Measurement Sets                                                   96
dtype: object

In [16]:
44*2

88

### Utils #4: convert aliases to accessions

In [5]:
with open("/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Buenrostro/buenrostro_bonemarrow_share-seq_20251002/aliases_buenrostro_bonemarrow_share-seq_20251002.txt") as f:
    list_of_aliases = [entry.strip() for entry in f.readlines()]

In [6]:
list_of_aliases

['buenrostro-bernstein:AnalysisSet_BioSam-4563_SS-PKR-1',
 'buenrostro-bernstein:AnalysisSet_BioSam-4564_SS-PKR-1',
 'buenrostro-bernstein:AnalysisSet_BioSam-4565_SS-PKR-1',
 'buenrostro-bernstein:AnalysisSet_BioSam-4566_SS-PKR-1',
 'buenrostro-bernstein:AnalysisSet_BioSam-4567_SS-PKR-1',
 'buenrostro-bernstein:AnalysisSet_BioSam-4568_SS-PKR-1',
 'buenrostro-bernstein:AnalysisSet_BioSam-4563_SS-PKR-2',
 'buenrostro-bernstein:AnalysisSet_BioSam-4564_SS-PKR-2',
 'buenrostro-bernstein:AnalysisSet_BioSam-4565_SS-PKR-2',
 'buenrostro-bernstein:AnalysisSet_BioSam-4566_SS-PKR-2',
 'buenrostro-bernstein:AnalysisSet_BioSam-4567_SS-PKR-2',
 'buenrostro-bernstein:AnalysisSet_BioSam-4568_SS-PKR-2',
 'buenrostro-bernstein:AnalysisSet_BioSam-4887_SS-PKR-212',
 'buenrostro-bernstein:AnalysisSet_BioSam-4563_SS-PKR-3',
 'buenrostro-bernstein:AnalysisSet_BioSam-4564_SS-PKR-3',
 'buenrostro-bernstein:AnalysisSet_BioSam-4565_SS-PKR-3',
 'buenrostro-bernstein:AnalysisSet_BioSam-4566_SS-PKR-3',
 'buenrostro

In [11]:
# Query portal
converted_accessions = []

all_relevant_anasets = igvf_client_api.analysis_sets(aliases=list_of_aliases).graph
for anaset in all_relevant_anasets:
    converted_accessions.append(anaset.accession)
    

In [14]:
# with open("/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/terra_datatables/setup/Buenrostro/buenrostro_bonemarrow_share-seq_20251002/buenrostro_bonemarrow_share-seq_20251002.txt", "w") as f:
#     for acc in converted_accessions:
#         f.writelines(f'{acc}\n')